<a href="https://colab.research.google.com/github/kfox-code/example-repo1/blob/main/neural_network_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project: Slogan Classifier and Generator

In this capstone project you will train a Long Short-Term Memory (LSTM) model to generate slogans for businesses based on their industry, and also train a classifier to predict the industry based on a given slogan.

##Libraries
We recommend running this notebook using [Google Colab](https://colab.google/) however if you choose to use your local machine you will need to install spaCy before starting.

To install spaCy, refer to the installation instructions provided on the spaCy [website](https://spacy.io/usage). Note you may need to install an older version of Python that is compatible with spaCy. You can create a virtual environment for this project to install the specific version of Python that you need.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
import spacy # available on Google Colab
from sklearn.model_selection import train_test_split

## Loading and viewing the dataset

- Load the slogan dataset into a variable called data.
- Extract relevant columns in a variable called df.
- Handle missing values.

Do **not** change the column names.

If you are using Google Colab you will need mount your Google Drive as follows:  
`from google.colab import drive`  
`drive.mount('/content/drive')`  

The path you use when loading your data will look something like this if you are using your Google Drive:  
"/content/drive/MyDrive/Colab Notebooks/slogan-valid.csv"

In [2]:
# Upload the dataset
from google.colab import files

uploaded = files.upload()

Saving slogan-valid.csv to slogan-valid.csv


In [3]:
# Load the dataset
data = pd.read_csv("slogan-valid.csv")
data.head()

,desc,output,type,company,industry,url,alias,desc_masked,output_masked,ent_dict,unsupported,first_pos
0,The latest <company> & Point of Sale tech for ...,Taking Care of Small Business Technology,headline_long,eftpos warehouse,computer hardware,eftposwarehouse.co.nz,Eftpos Warehouse,The latest <company> & Point of Sale tech for ...,Taking Care of Small Business Technology,{'[date]': 'monthly'},False,VB
1,Easily deliver personalized activities that en...,Build World-Class Recreation Programs,headline,welbi,"health, wellness and fitness",welbi.co,Welbi,Easily deliver personalized activities that en...,Build World-Class Recreation Programs,{},False,VB
2,Powerful lead generation software that convert...,Most Powerful Lead Generation Software for Mar...,headline_long,optinmonster,internet,optinmonster.com,Optinmonster,Powerful lead generation software that convert...,Most Powerful Lead Generation Software for Mar...,{},False,JJ
3,Twine matches companies to the best digital an...,Hire quality freelancers for your job,headline_long,twine.fm,internet,twine.fm,NaN,Twine matches companies to the best digital an...,Hire quality freelancers for your job,"{'[number]': 'over 260,000'}",False,VB
4,"Financial Advisers Norwich, Norfolk - <company...","Financial Advisers Norwich, Norfolk",headline,mcb financial services ltd,financial services,mcbfinancialservices.co.uk,Mcb Financial Services,"Financial Advisers [country], [country1] - <co...","Financial Advisers [country], [country1]","{'[country]': 'Norwich', '[country1]': 'Norfolk'}",False,NN


In [4]:
# Name your pandas DataFrame 'df'
df = data[['output', 'industry']]
# Hande missing values
df = df.dropna()

## Data Preprocessing

Since we are working with textual data, we need software that understands natural language. For this, we'll use a library for processing text called **spaCy**. Using spaCy, we'll break the text into smaller units called tokens that are easier for the machine to process. This process is called **tokenisation**. We'll also convert all text to lowercase and remove punctuation because this information is not necessary for our models. Run the code below, and your dataframe (df) will gain a new column called **'processed_slogan'** which contains the preprocessed text.




In [5]:
# Load spaCy model for text processing
nlp = spacy.load("en_core_web_sm")

# Define text preprocessing function
def preprocess_text(text):
    text_lower = text.lower()
    doc = nlp(text_lower)

    processed_tokens = []

    for token in doc:
        if not token.is_punct:
            processed_tokens.append(token.text)

    return " ".join(processed_tokens)

df["processed_slogan"] = df["output"].apply(preprocess_text)

df.head()

,output,industry,processed_slogan
0,Taking Care of Small Business Technology,computer hardware,taking care of small business technology
1,Build World-Class Recreation Programs,"health, wellness and fitness",build world class recreation programs
2,Most Powerful Lead Generation Software for Mar...,internet,most powerful lead generation software for mar...
3,Hire quality freelancers for your job,internet,hire quality freelancers for your job
4,"Financial Advisers Norwich, Norfolk",financial services,financial advisers norwich norfolk


We want our model to generate **industry-specific** slogans. If we use the 'processed_slogan' column as it is, we'll be leaving out crucial context - the industries of the companies behind those slogans. To fix this, we'll create a new **'modified_slogan'** column that adds the industry name to the front of processed slogan.  

For example:  

> industry = 'computer hardware'  
processed_slogan = 'taking care of small business technology'  
modified_slogan = 'computer hardware taking care of small business technology'

Write code in the cell below to achieve this.

In [6]:
df["modified_slogan"] = df["industry"] + " " + df["processed_slogan"]

Now we need to get data to train our model. We have textual data which we will need to represent numerically for our model to learn from it.  
The code below does the following:
1. Tokenizes a dataset of slogans.
2. Converts words to numerical indices.
3. Creates input sequences using the numerical indices.  

Here's how it works. From the 'modified_slogan' column, we take the slogan "computer hardware taking care of small business technology". The tokenisation process will convert words into their corresponding indices:  

<center>

| Word         | Token Index |
|-------------|-------|
| "computer"  | 1     |
| "hardware"  | 2     |
| "taking"    | 3     |
| "care"      | 4     |
| "of"        | 5     |
| "small"     | 6     |
| "business"  | 7     |
| "technology"| 8     |

</center>

So the tokenized list is:

<center>
[1, 2, 3, 4, 5, 6, 7, 8]
</center>

When creating input sequences for training, the loop generates progressively longer sequences.

<center>

| Token Index Sequence               | Corresponding Slogan                                 |
|------------------------------|-----------------------------------------------------|
| [1, 2]                       | "computer hardware"                                |
| [1, 2, 3]                    | "computer hardware taking"                        |
| [1, 2, 3, 4]                 | "computer hardware taking care"                   |
| [1, 2, 3, 4, 5]              | "computer hardware taking care of"                |
| [1, 2, 3, 4, 5, 6]           | "computer hardware taking care of small"          |
| [1, 2, 3, 4, 5, 6, 7]        | "computer hardware taking care of small business" |
| [1, 2, 3, 4, 5, 6, 7, 8]     | "computer hardware taking care of small business technology" |

</center>

Instead of training the model on only **complete slogans**, we provide partial phrases which will help the model learn how words connect over time. This will make it better at predicting the next word when generating slogans.  

Run the cell block below to generate the input sequences. Be sure to read the comments to understand what the code is doing.


In [7]:
'''** Clean up comments'''

# Tokenizer to convert words into numerical values tokens
tokenizer = Tokenizer()

# Tokenizer learns words in dataset
tokenizer.fit_on_texts(df["modified_slogan"])

# Total number of unique words in learned vocabulary
total_words = len(tokenizer.word_index) + 1

# Dictionary mapping words to its numerical index: index based on frequency i.e., more freq => lower index
tokenizer.word_index

# Creating input sequences
# Initialise list to store the input sequences
input_sequences = []

# Iterate over processed slogans
for line in df["modified_slogan"]:

    # Convert slogans to token sequences
    token_list = tokenizer.texts_to_sequences([line])[0] # returns list containing list of words indices; extracting inner list [0]

    # token_list is a list of tokenized word INDICES
    # Building list of progressively longer input sequences for better training
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1])

The input sequences created above are of **varying lengths**, which will be a problem when training our LSTM model. LSTMs require input sequences of **equal length**. So, we need to **pad** shorter sequences by **prepending zeros** until they match the length of the longest sequence.  

For example, if the longest sequence has **10 tokens**, our padded sequences will look like this:

<center>

| Input Sequence                     | Padded Sequence                         |
|-------------------------------------|-----------------------------------------|
| [1, 2]                              | [0, 0, 0, 0, 0, 0, 0, 0, 1, 2]         |
| [1, 2, 3]                           | [0, 0, 0, 0, 0, 0, 0, 1, 2, 3]         |
| [1, 2, 3, 4]                        | [0, 0, 0, 0, 0, 0, 1, 2, 3, 4]         |
| [1, 2, 3, 4, 5]                     | [0, 0, 0, 0, 0, 1, 2, 3, 4, 5]         |
| [1, 2, 3, 4, 5, 6]                  | [0, 0, 0, 0, 1, 2, 3, 4, 5, 6]         |
| [1, 2, 3, 4, 5, 6, 7]               | [0, 0, 0, 1, 2, 3, 4, 5, 6, 7]         |
| [1, 2, 3, 4, 5, 6, 7, 8]            | [0, 0, 1, 2, 3, 4, 5, 6, 7, 8]         |

</center>

In the cell below, write code that **finds the length of the longest sequence** in **input_sequences** and stores this value in a variable named **max_seq_len**.


In [8]:
# Find the length of the longest input sequence
max_seq_len = max(len(seq) for seq in input_sequences)

max_seq_len

15

Run the cell below to pad the input sequences so they are all the same length as **max_seq_length**.

In [9]:
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding="pre")

## Training Data for Slogan Generator

The input sequences generated will be used as our training data. Our LSTM needs to learn how to predict the **next word** in a sequence.  

The inputs for our model will be the input sequences **excluding the last token index** and the outputs will be the **last token index**.  

As an example, let us use the input sequence [0, 0, 1, 2, 3, 4, 5, 6, 7, 8] and say it corresponds to the slogan "computer hardware taking care of small business technology". When training the model:

> Our input **x** will be the input sequence [0, 0, 1, 2, 3, 4, 5, 6, 7] corresponding to "computer hardware taking care of small".  
> Our output **y** will be [8] which corresponds to "business".  

In the code cell below, use `input_sequences` to create the following two variables:
1. **X_gen** which contains the input sequences excluding the last token index.
2. **y_gen** which contains the last token index of the input sequence.

In [10]:
# Input sequences (all tokens except the last one)
X_gen = input_sequences[:, :-1]

# Output tokens (last token of each sequence)
y_gen = input_sequences[:, -1]

The model will output the next word of a sequence over a probability distribution. We need to encode our output variable for this to be possible.

In the code cell below, write code that will apply one-hot encoding to **y_gen** using `tf.keras.utils.to_categorical()`. **Maintain the same variable name**.  

*Hint: set the `num_classes` (number of classes) parameter to the total number of unique words in the learned vocabulary. You can access this value through a variable that was created when generating input sequences earlier.*

In [11]:
# One-hot encode the output variable
y_gen = tf.keras.utils.to_categorical(y_gen, num_classes=total_words)

## Slogan Generator Architecture

In the code cell that follows, configure the LSTM following these steps:

1. Create a sequential model using `tf.keras.models.Sequential()`. This model will have an embedding layer, two LSTM layers, and a dense output layer.
2. Add an embedding layer that converts words into dense vector representations. This layer should:
> *   Have `total_words`as the vocabulary size.
> *   Use 100 as an embedding dimension.
> *   Takes an input length of `max_seq_len - 1` (excludes the target word).
3. Add two LSTM layers.
> *   The first LSTM layer should have 150 **and** set `return_sequences` to `True`.
> *   The second LSTM layer should have 100 units.
4. Add a dense output layer which:
> *   Uses `total_words` as the number of units (one for each word in the vocabulary).
> *   Uses a softmax activation function.
5. Use `Sequential` to put everything together in the correct order to complete the architecture of the LSTM model called **gen_model**.


In [13]:
# Create the LSTM model for slogan generation
gen_model = Sequential()

# Embedding layer
gen_model.add(
    Embedding(
        input_dim=total_words,
        output_dim=100
    )
)

# First LSTM layer (returns sequences)
gen_model.add(
    LSTM(
        150,
        return_sequences=True
    )
)

# Second LSTM layer
gen_model.add(
    LSTM(
        100
    )
)

# Dense output layer with softmax activation
gen_model.add(
    Dense(
        total_words,
        activation="softmax"
    )
)

In the code cell below, compile `gen_model` using `categorical_crossentropy` loss, an Adam optimiser, and an appropriate metric of your choice.


In [14]:
gen_model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(),
    metrics=["accuracy"]
)

## Slogan Generation

In the code cell below, fit the compiled model on the inputs and outputs, setting the **number of epochs to 50**.

In [15]:
gen_model.fit(
    X_gen,
    y_gen,
    epochs=50,
    verbose=1
)

Epoch 1/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 78s 66ms/step - accuracy: 0.0626 - loss: 7.3605
Epoch 2/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 72s 66ms/step - accuracy: 0.0960 - loss: 6.4085
Epoch 3/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 70s 64ms/step - accuracy: 0.1350 - loss: 6.0419
Epoch 4/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 82s 64ms/step - accuracy: 0.1694 - loss: 5.7347
Epoch 5/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 70s 64ms/step - accuracy: 0.1891 - loss: 5.4496
Epoch 6/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 71s 66ms/step - accuracy: 0.2053 - loss: 5.2125
Epoch 7/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 70s 64ms/step - accuracy: 0.2306 - loss: 4.9637
Epoch 8/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 70s 64ms/step - accuracy: 0.2466 - loss: 4.7398
Epoch 9/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 71s 65ms/step - accuracy: 0.2533 - loss: 4.5595
Epoch 10/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 71s 66ms/step - accuracy: 0.2644 - loss: 4.3667
Epoch 11/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 70s 65ms/step - accuracy: 0.2725 - loss: 4.1940
Epoch 12

We will now define a function called `generate_slogan` which will generate a slogan by predicting one word at a time based on a given starting phrase (the `seed_text`). This function will do this using our trained model, `gen_model`.

Here is a breakdown of how the algorithm works:  

Let us assume the dictionary mapping words to unique indices, `tokenizer.word_index`, looks like this:

> `{'computer': 1, 'hardware': 2, 'taking': 3, 'care': 4, 'of': 5}`

If the model's predicted index for the next word is 3 (`predicted_index = 3`), the loop will:

> Check 'computer' (index 1) → No match  
> Check 'hardware' (index 2) → No match  
> Check 'taking' (index 3) → Match found!  
> Assign output_word = "taking" and exit the loop.  

The `output_word` will be appended to the `seed_text`, and the process will continue to add words to the `seed_text` until we have reached the maximum number of words **or** an invalid prediction occurs.  

Carefully follow the code below and complete the missing parts as guided by the comments.

In [16]:
def generate_slogan(seed_text, max_words=20):
    for _ in range(max_words):

        # Tokenising and padding seed_text
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding="pre")

        # Use your trained model (gen_model) on token_list to predict the probability distribution of the next word over the vocabulary
        predictions = gen_model.predict(token_list, verbose=0)

        # From the predicted probabilities (stored in the variable predictions), identify the word index with the highest probability using np.argmax()
        predicted_index = np.argmax(predictions, axis=-1)[0]

        output_word = None

        # Searching for the word that corresponds to the predicted index
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break

        # If no valid word is found, algorithm stops
        if output_word is None:
            break # out of main loop

        # If valid a word is found, we append the predicted word to seed_text and continue the process until max_words is reached or an invalid prediction occurs
        # Complete the code below to add a word to seed_text
        seed_text = seed_text + " " + output_word

    return seed_text

## Training Data for Slogan Classifier

We will now prepare the data we will use to train our classifier. For our classifier, the inputs will come from the `processed_slogans` column of our DataFrame, `df`. The outputs will be the different industry categories under the `industry` column.

In the code cell below, extract the unique values from the `industry` column in the DataFrame and store these in a variable called **industries**.

In [17]:
industries = df["industry"].unique()

Create a dictionary called `industry_to_index` where each unique industry is mapped to a unique index starting from 0.

*Hint: Use the `enumerate()` function.*

In [18]:
industry_to_index = {industry: index for index, industry in enumerate(industries)}

Create a new column `industry_index` in your DataFrame by mapping the `industry` column to the indices using the `industry_to_index` dictionary.

*Hint: Use the  `map()` function.*

In [19]:
df["industry_index"] = df["industry"].map(industry_to_index)

Split the DataFrame `df` into training and testing sets, setting aside 20% of the data for the test set. Be sure to set the parameter `stratify=df["industry_index"]`. This ensures that both sets have the same proportion of each class (industry) as in the original dataset, resulting in balanced datasets. Call the training DataFrame `df_train` and the testing DataFrame `df_test`.

In [21]:
# Remove industries that have fewer than 2 examples (required for stratified split)
class_counts = df["industry_index"].value_counts()
valid_classes = class_counts[class_counts >= 2].index

df = df[df["industry_index"].isin(valid_classes)].copy()

In [22]:
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    stratify=df["industry_index"],
    random_state=42
)

Our classifier will use padded slogan sequences as inputs, similar to input sequences used for the slogan generator. The difference is we will not use sequences that get progressively longer, but instead we will use **complete slogans**. This is because our classifier does not need to learn how to predict what word comes next. It needs the full context of a slogan to learn how to accurately predict the industry.  

The next steps will walk you through how to create these sequences.  

We previously created and fitted a `Tokenizer` object called `tokenizer` while preparing data for the slogan generator. Now, we will reuse it to convert words into numerical indices.  

In the code cell below, use the `texts_to_sequences()` **method** of `tokenizer` to transform the `processed_slogan` column in **both** the `df_train` and `df_test` DataFrames into sequences of numerical indices. Store the results in variables named `X_train` and `X_test`.


In [23]:
# Convert processed slogans to sequences for training and testing
X_train = tokenizer.texts_to_sequences(df_train["processed_slogan"])
X_test = tokenizer.texts_to_sequences(df_test["processed_slogan"])

The slogan sequences are of varying lengths. We will need to pad them the same way we did to the input sequences for the slogan generator. The `pad_sequences()` function can ensure the sequences in `slogan_sequences` have the same length.  

In the code cell below, use the `pad_sequences()` function to standardise the `slogan_sequences` lengths. Set the `maxlen` parameter to `max_seq_len`, the `padding` parameter to 0, and assign the resulting padded sequences to the same variables, `X_train` and `X_test`.

In [24]:
# Pad slogan sequences to the same length
X_train = pad_sequences(
    X_train,
    maxlen=max_seq_len,
    padding="pre",
    value=0
)

X_test = pad_sequences(
    X_test,
    maxlen=max_seq_len,
    padding="pre",
    value=0
)

We have successfully created training and testing inputs for our model. Now, we will create the outputs - industry categories.

 In the code cell that follows, use `tf.keras.utils.to_categorical()` to apply one-hot encoding to the `industry_index` column of **both** `df_train` and `df_test` DataFrames. Assign the results to a variables named `y_train` and `y_test`.

 *Hint: set the `num_classes` parameter to the total number of industries in the DataFrame. The `industries` variable can be used to find this value.*

In [25]:
# One-hot encode industry labels for training and testing
y_train = tf.keras.utils.to_categorical(
    df_train["industry_index"],
    num_classes=len(industries)
)

y_test = tf.keras.utils.to_categorical(
    df_test["industry_index"],
    num_classes=len(industries)
)

## Slogan Classifier Architecture

Configure the LSTM classifier following these steps:  


1. Create a Sequential model:  
   Use `tf.keras.models.Sequential()` to create a sequential model. This model will consist of an embedding layer, two LSTM layers, and a dense output layer.

2. Add an embedding layer which will convert words into dense vector representations. Configure this layer with:
   > * `total_words` as the vocabulary size.
   > * 100 as the embedding dimension.
   > * `max_seq_len` as the `input_length` (this is the length of the slogans).

3. Add the first LSTM layer. Configure it with:
   > * 150 units.
   > * Set `return_sequences` to `True` to ensure the layer outputs sequences for the next LSTM layer.

4. Add the second LSTM layer which will process the output from the previous LSTM layer. Configure it with:
   > * 100 units.
   > * No need to set `return_sequences` here (it is the final LSTM layer).

5. Add the dense output layer which will classify the data into industries. Configure it with:
   > * The number of unique industries as the number of units.
   > * The `softmax` activation function to get probabilities for each class (industry).

6. Use `Sequential` to arrange all layers in the correct order and complete the architecture of the LSTM model called **class_model**.


In [27]:
# Create the LSTM model for slogan classification
class_model = Sequential()

# Embedding layer
class_model.add(
    Embedding(
        input_dim=total_words,
        output_dim=100,
    )
)

# First LSTM layer
class_model.add(
    LSTM(
        150,
        return_sequences=True
    )
)

# Second LSTM layer
class_model.add(
    LSTM(
        100
    )
)

# Dense output layer (number of industries)
class_model.add(
    Dense(
        len(industries),
        activation="softmax"
    )
)

In the code cell below, compile `class_model` using `categorical_crossentropy` loss, an Adam optimiser, and an appropriate metric of your choice.

In [28]:
class_model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(),
    metrics=["accuracy"]
)

## Slogan Classification & Evaluation

In the code cell that follows, fit the compiled model on the inputs and outputs, setting **the number of epochs to 50**.

In [29]:
class_model.fit(
    X_train,
    y_train,
    epochs=50,
    verbose=1
)

Epoch 1/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 12s 61ms/step - accuracy: 0.0648 - loss: 4.5423
Epoch 2/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 10s 61ms/step - accuracy: 0.0806 - loss: 4.2960
Epoch 3/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.0812 - loss: 4.2879
Epoch 4/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.1013 - loss: 4.0171
Epoch 5/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.1676 - loss: 3.6705
Epoch 6/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.2477 - loss: 3.3057
Epoch 7/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.3496 - loss: 2.8695
Epoch 8/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 66ms/step - accuracy: 0.4329 - loss: 2.4872
Epoch 9/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 60ms/step - accuracy: 0.5074 - loss: 2.1458
Epoch 10/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.5576 - loss: 1.8172
Epoch 11/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.6329 - loss: 1.5841
Epoch 12/50
134/134 ━━━━━━━━━━━━━━━━━━━

Evaluate the model using the testing set. Add a comment on the model's performance.

In [30]:
# Evaluate the classifier on the test set
test_loss, test_accuracy = class_model.evaluate(X_test, y_test, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


Test Loss: 7.6793
Test Accuracy: 0.1845


The LSTM classifier achieved a test accuracy of approximately 18.5%, with a categorical cross-entropy loss of 7.68 on the test set. This level of accuracy indicates that the model performs only slightly better than random guessing, suggesting limited generalisation to unseen slogans. Performance could potentially be improved by increasing dataset size, applying regularisation techniques, or experimenting with more advanced architectures.

We will now define a function called `classify_slogan` which takes a slogan as input and predicts the industry it belongs to using the trained model, `class_model`.  

Carefully follow the code below and complete the missing parts (indicated by ellipses) as guided by the comments.

In [31]:
def classify_slogan(slogan):
    # Use the preprocess_text to clean the input slogan
    slogan = preprocess_text(slogan)

    # Converting the slogan to a sequence of indices
    sequence = tokenizer.texts_to_sequences([slogan])

    # Pad the sequence using the pad_sequences() function
    padded_sequence = pad_sequences(
        sequence,
        maxlen=max_seq_len,
        padding="pre",
        value=0
    )

    # Pass padded_sequence into the class_model to get the predicted probabilities
    prediction = class_model.predict(padded_sequence, verbose=0)

    # Get the index of the industry with the highest probability
    predicted_index = np.argmax(prediction, axis=-1)[0]

    # Return the predicted industry
    return industries[predicted_index]

## Combining the two models

Run the code cell below to combine the two models: we will first generate a slogan for a company in the "internet" industry, then pass the generated slogan to the slogan classifier to see if it correctly classifies it as internet.

In [32]:
industry = "internet"
generated_slogan = generate_slogan(industry)
predicted_industry = classify_slogan(generated_slogan)

print(f"Generated Slogan: {generated_slogan}")
print(f"Predicted Industry: {predicted_industry}")

Generated Slogan: internet web design digital marketing agency newcastle estate drupal applications and risk management in yorkshire falls bangkok games companies tv family
Predicted Industry: computer software


Compare the results and comment on any differences you notice between the generated slogans and the classifier’s predictions in the markdown cell below.


The slogan generator produced a phrase containing terms such as web design, digital marketing, drupal, and applications. The classifier predicted computer software, which is a plausible because these terms are strongly associated with software/technology industries. Since the classifier achieved only 18.45% test accuracy, misclassifications are expected.